In [1]:
# pip install StackAPI
# Save keys in stackexchange.json
# Docs are here: https://stackapi.readthedocs.io/en/latest/
# https://api.stackexchange.com/docs


In [28]:
import pandas as pd
import numpy as np
import json 
from datetime import datetime
from stackapi import StackAPI

In [29]:
key_file = "./stackexchange.json"
keys = json.load(open(key_file,"r"))

In [97]:
SITE = StackAPI("robotics")
start_day_this = datetime(2023,1,1)
stop_day_this = datetime(2023,12,8)
start_day_last = datetime(2022,1,1)
stop_day_last = datetime(2022,12,31)
this_year = 2023
last_year = 2022

In [44]:
# Get all questions tagged "ROS" in 2023
ros_questions = SITE.fetch('questions', fromdate=start_day, todate=stop_day, tagged='ros')

In [45]:
print(len(ros_questions["items"]))

441


In [98]:
def fetch_results(start_day,stop_day,tags):
    full_results = []
    for i in range(1,10):
        temp = SITE.fetch('questions', fromdate=start_day, todate=stop_day, tagged=tags, page=i)
        fetched = len(temp["items"])
        
        full_results += temp["items"]
        if fetched < 500:
            break
    print("{0} questions tagged with {1} between {2} and {3} ".format(len(full_results),tags,start_day,stop_day))
    return full_results

In [103]:
# TODO purge "ROS2" questions from "ROS" set

ros_this = fetch_results(start_day_this,stop_day_this,"ros")
ros1_this = fetch_results(start_day_this,stop_day_this,"ros1")
ros2_this = fetch_results(start_day_this,stop_day_this,"ros2")
ros_last = fetch_results(start_day_last,stop_day_last,"ros")
ros1_last = fetch_results(start_day_last,stop_day_last,"ros1")
ros2_last = fetch_results(start_day_last,stop_day_last,"ros2")

440 questions tagged with ros between 2023-01-01 00:00:00 and 2023-12-08 00:00:00 
0 questions tagged with ros1 between 2023-01-01 00:00:00 and 2023-12-08 00:00:00 
1980 questions tagged with ros2 between 2023-01-01 00:00:00 and 2023-12-08 00:00:00 
1408 questions tagged with ros between 2022-01-01 00:00:00 and 2022-12-31 00:00:00 
0 questions tagged with ros1 between 2022-01-01 00:00:00 and 2022-12-31 00:00:00 
386 questions tagged with ros2 between 2022-01-01 00:00:00 and 2022-12-31 00:00:00 


In [121]:
# we're ignoring the "ros1" tag and assuming the tag "ros" == ROS 1 
ros_q_this = len(ros_this) + len(ros2_this)
ros1_q_this = len(ros_this)
ros2_q_this = len(ros2_this)
ros_q_last = len(ros_last) + len(ros2_last)
ros1_q_last = len(ros_last)
ros2_q_last = len(ros2_last)
print("In {0} there were {1} ROS questions asked.".format(this_year,ros_q_this))
print("In {0} there were {1} ROS questions asked.".format(last_year,ros_q_last))
total_change = (ros_q_this-ros_q_last) / ros_q_last
print("In {0} there was a {1:+.2f}% change in ROS questions asked.".format(this_year,total_change*100.0))
this_prct_ros2 = ros2_q_this / ros_q_this
last_prct_ros2 = ros2_q_last / ros_q_last
print("In {0} {1:.2f}% of questions were ROS 2 and {2:.2f}% were ROS 1 ".format(this_year,this_prct_ros2*100.0,100.0-(this_prct_ros2*100.0)))
print("In {0} {1:.2f}% of questions were ROS 2 and {2:.2f}% were ROS 1 ".format(last_year,last_prct_ros2*100.0,100.0-(last_prct_ros2*100.0)))

In 2023 there were 2420 ROS questions asked.
In 2022 there were 1794 ROS questions asked.
In 2023 there was a +34.89% change in ROS questions asked.
In 2023 81.82% of questions were ROS 2 and 18.18% were ROS 1 
In 2022 21.52% of questions were ROS 2 and 78.48% were ROS 1 


In [139]:
from collections import Counter

def summarize_tags(questions):
    summary = {}
    summary["tags"] = [] 
    summary["answers"] = 0
    summary["is_answered"] = 0
    summary["views"] = 0
    summary["scores"] = []
    summary["reputations"] = [] 
    for i in questions:
        summary["tags"] += i["tags"]
        summary["scores"].append(i["score"])
        summary["answers"] += i["answer_count"]
        if i["is_answered"]:
            summary["is_answered"] += 1
        summary["views"] += i["view_count"]
        if "reputation" in i["owner"]:
            summary["reputations"].append(i["owner"]["reputation"])
    summary["tags"] = Counter(summary["tags"])
    summary["total"] = len(questions)
    summary["prct_answered"] = summary["is_answered"] / len(questions)
    summary["total_answers"] = np.sum(summary["answers"])
    summary["total_views"] = np.sum(summary["views"])
    summary["median_reputation"] = np.median(summary["reputations"])
    return summary

In [157]:
ros_this_summary = summarize_tags(ros_this)
ros2_this_summary = summarize_tags(ros2_this)
all_ros_this_summary = summarize_tags(ros_this+ros2_this)
ros_last_summary = summarize_tags(ros_last)
ros2_last_summary = summarize_tags(ros2_last)
all_ros_last_summary = summarize_tags(ros_last+ros2_last)
names = ["ros_this","ros_last","ros2_this","ros2_last","all_this","all_last"]
dsets = [ros_this_summary,ros_last_summary,ros2_this_summary,ros2_last_summary,all_ros_this_summary,all_ros_last_summary]
# Note that ROS2 is often in the set of ROS questions, so we probably should purge those questions so we don't get an overcount
for name, dset in zip(names,dsets):
    print("{0}: {1}".format(name,dset["tags"].most_common()[1:10] ))


ros_this: [('ros2', 91), ('ros-noetic', 53), ('gazebo', 44), ('rviz', 41), ('moveit', 28), ('navigation', 26), ('ros-melodic', 24), ('ros-humble', 19), ('python', 19)]
ros_last: [('ros2', 328), ('ros-melodic', 228), ('navigation', 95), ('gazebo', 86), ('moveit', 78), ('rviz', 75), ('ros-kinetic', 55), ('tf2', 48), ('transform', 45)]
ros2_this: [('ros-humble', 365), ('gazebo', 221), ('ros', 197), ('nav2', 161), ('rviz', 102), ('ros-foxy', 88), ('python', 86), ('navigation', 83), ('moveit', 79)]
ros2_last: [('ros', 144), ('navigation', 22), ('roslaunch', 19), ('c++', 16), ('python', 16), ('gazebo', 12), ('colcon', 11), ('rviz', 11), ('docker', 9)]
all_this: [('ros', 637), ('ros-humble', 384), ('gazebo', 265), ('nav2', 164), ('rviz', 143), ('navigation', 109), ('moveit', 107), ('python', 105), ('ros-foxy', 89)]
all_last: [('ros2', 714), ('ros-melodic', 228), ('navigation', 117), ('gazebo', 98), ('rviz', 86), ('moveit', 84), ('python', 55), ('ros-kinetic', 55), ('c++', 53)]
